In [23]:
import sys  # Remove in production - KTZ

sys.path.append("..")  # Remove in production - KTZ
import pandas as pd
import datetime
from PolygonTickData.Helper import Helper
from CalculateETFArbitrage.LoadEtfHoldings import LoadHoldingsdata
from CalculateETFArbitrage.GatherData import DataApi
import numpy as np

# Jupyter Notebook to display all the columns in a df
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', -1)  # or 199

In [10]:
etfname = 'XLK'
date = '2020-03-17'

etfData = LoadHoldingsdata().LoadHoldingsAndClean(etfname=etfname, fundholdingsdate=date)
allData = DataApi(etfname=etfname, date=date, etfData=etfData)

DEBUG:root:Command createIndexes with request id 52326674 started on server ('52.91.26.227', 27017)
DEBUG:root:Command createIndexes with request id 52326674 on server ('52.91.26.227', 27017) succeeded in 164426 microseconds
DEBUG:root:Command find with request id 1133598295 started on server ('52.91.26.227', 27017)
DEBUG:root:Command find with request id 1133598295 on server ('52.91.26.227', 27017) succeeded in 285272 microseconds
DEBUG:root:Command endSessions with request id 2059111528 started on server ('52.91.26.227', 27017)


ETF object
2020-03-17 00:00:00


DEBUG:root:Command endSessions with request id 2059111528 on server ('52.91.26.227', 27017) succeeded in 676785 microseconds
INFO:root:Data Successfully Loaded
DEBUG:root:Command count with request id 788479691 started on server ('localhost', 27017)
DEBUG:root:Command count with request id 788479691 on server ('localhost', 27017) succeeded in 1173 microseconds
DEBUG:root:Command count with request id 2004064647 started on server ('localhost', 27017)
DEBUG:root:Command count with request id 2004064647 on server ('localhost', 27017) succeeded in 1307 microseconds
DEBUG:root:Command count with request id 1181002581 started on server ('localhost', 27017)
DEBUG:root:Command count with request id 1181002581 on server ('localhost', 27017) succeeded in 2082 microseconds
DEBUG:root:Command count with request id 2066513293 started on server ('localhost', 27017)
DEBUG:root:Command count with request id 2066513293 on server ('localhost', 27017) succeeded in 2863 microseconds
DEBUG:root:Command cou

DEBUG:root:Command count with request id 438285377 started on server ('localhost', 27017)
DEBUG:root:Command count with request id 438285377 on server ('localhost', 27017) succeeded in 1006 microseconds
DEBUG:root:Command count with request id 393422029 started on server ('localhost', 27017)
DEBUG:root:Command count with request id 393422029 on server ('localhost', 27017) succeeded in 877 microseconds
DEBUG:root:Command count with request id 141892290 started on server ('localhost', 27017)
DEBUG:root:Command count with request id 141892290 on server ('localhost', 27017) succeeded in 2247 microseconds
DEBUG:root:Command count with request id 1076869860 started on server ('localhost', 27017)
DEBUG:root:Command count with request id 1076869860 on server ('localhost', 27017) succeeded in 1412 microseconds
DEBUG:root:Command count with request id 2107043751 started on server ('localhost', 27017)
DEBUG:root:Command count with request id 2107043751 on server ('localhost', 27017) succeeded in 

Fetching Data


...

-------0.3614320755004883-------
Fetching Data


DEBUG:root:Command aggregate with request id 586004101 on server ('localhost', 27017) succeeded in 12933668 microseconds
DEBUG:root:Command getMore with request id 1167620774 started on server ('localhost', 27017)
DEBUG:root:Command getMore with request id 1167620774 on server ('localhost', 27017) succeeded in 899995 microseconds
DEBUG:root:Command getMore with request id 496782332 started on server ('localhost', 27017)
DEBUG:root:Command getMore with request id 496782332 on server ('localhost', 27017) succeeded in 822055 microseconds
DEBUG:root:Command getMore with request id 4234388 started on server ('localhost', 27017)
DEBUG:root:Command getMore with request id 4234388 on server ('localhost', 27017) succeeded in 844496 microseconds
DEBUG:root:Command getMore with request id 300398765 started on server ('localhost', 27017)
DEBUG:root:Command getMore with request id 300398765 on server ('localhost', 27017) succeeded in 877840 microseconds
DEBUG:root:Command getMore with request id 67

DEBUG:root:Command count with request id 1745596560 on server ('localhost', 27017) succeeded in 1459 microseconds
DEBUG:root:Command count with request id 1467282253 started on server ('localhost', 27017)
DEBUG:root:Command count with request id 1467282253 on server ('localhost', 27017) succeeded in 4691 microseconds
DEBUG:root:Command count with request id 1058107670 started on server ('localhost', 27017)
DEBUG:root:Command count with request id 1058107670 on server ('localhost', 27017) succeeded in 904 microseconds
DEBUG:root:Command count with request id 303528883 started on server ('localhost', 27017)
DEBUG:root:Command count with request id 303528883 on server ('localhost', 27017) succeeded in 916 microseconds
DEBUG:root:Command count with request id 1136274956 started on server ('localhost', 27017)
DEBUG:root:Command count with request id 1136274956 on server ('localhost', 27017) succeeded in 1724 microseconds
DEBUG:root:Command count with request id 1948596368 started on server 

-------32.89865708351135-------


DEBUG:root:Command count with request id 621699896 started on server ('localhost', 27017)
DEBUG:root:Command count with request id 621699896 on server ('localhost', 27017) succeeded in 1208 microseconds
DEBUG:root:Command count with request id 1402209417 started on server ('localhost', 27017)
DEBUG:root:Command count with request id 1402209417 on server ('localhost', 27017) succeeded in 1629 microseconds
DEBUG:root:Command count with request id 448129341 started on server ('localhost', 27017)
DEBUG:root:Command count with request id 448129341 on server ('localhost', 27017) succeeded in 1220 microseconds
DEBUG:root:Command count with request id 484684158 started on server ('localhost', 27017)
DEBUG:root:Command count with request id 484684158 on server ('localhost', 27017) succeeded in 880 microseconds
DEBUG:root:Command count with request id 681170435 started on server ('localhost', 27017)
DEBUG:root:Command count with request id 681170435 on server ('localhost', 27017) succeeded in 41

In [11]:
helperObj = Helper()
allData.tradesDataDf['Time'] = allData.tradesDataDf['Time'].apply(lambda x: helperObj.getHumanTime(ts=x, divideby=1000))
tradePricesDFMinutes = allData.tradesDataDf.groupby([allData.tradesDataDf['Time'], allData.tradesDataDf['Symbol']])['Trade Price']
tradePricesDFMinutes = tradePricesDFMinutes.first().unstack(level=1)

priceforNAVfilling = allData.openPriceData.set_index('Symbol').T.to_dict('records')[0]

In [12]:
marketStartTime = datetime.datetime.strptime('13:29:00', "%H:%M:%S").time()
marketEndTime = datetime.datetime.strptime('20:00:00', "%H:%M:%S").time()

mask = (tradePricesDFMinutes.index.time >= marketStartTime) & (tradePricesDFMinutes.index.time <= marketEndTime)
tradePricesDFMinutes=tradePricesDFMinutes[mask]
tradePricesDFMinutes=tradePricesDFMinutes.ffill(axis=0)
tradePricesDFMinutes = tradePricesDFMinutes.fillna(priceforNAVfilling)

etfprice = tradePricesDFMinutes[etfname]
tradePricesDFMinutes = tradePricesDFMinutes.pct_change().dropna() * 100

etfpricechange = tradePricesDFMinutes[etfname]
del tradePricesDFMinutes[etfname]

In [13]:
allData.quotesDataDf['Time'] = allData.quotesDataDf['Time'].apply(lambda x: helperObj.getHumanTime(ts=x, divideby=1000000000))
allData.quotesDataDf['Time'] = allData.quotesDataDf['Time'].map(lambda x: x.replace(second=0))
allData.quotesDataDf = allData.quotesDataDf[allData.quotesDataDf['Bid Size'] != 0]
allData.quotesDataDf = allData.quotesDataDf[allData.quotesDataDf['Ask Size'] != 0]
allData.quotesDataDf['Total Bid Ask Size'] = allData.quotesDataDf['Ask Size'] + allData.quotesDataDf['Bid Size']
allData.quotesDataDf['Spread'] = allData.quotesDataDf['Ask Price'] - allData.quotesDataDf['Bid Price']
allData.quotesDataDf['MidPrice'] = (allData.quotesDataDf['Ask Price'] + allData.quotesDataDf['Bid Price']) / 2
quotesSpreadsMinutes = allData.quotesDataDf.groupby('Time')['Spread'].mean()

In [14]:
del allData

In [7]:
netassetvaluereturn = tradePricesDFMinutes.assign(**etfData.getETFWeights()).mul(tradePricesDFMinutes).sum(axis=1)
ds = pd.concat([etfprice, etfpricechange, netassetvaluereturn, quotesSpreadsMinutes], axis=1).dropna()
ds.columns = ['ETF Price', 'ETF Change Price %', 'Net Asset Value Change%', 'ETF Trading Spread in $']
ds['Arbitrage in $'] = (ds['ETF Change Price %'] - ds['Net Asset Value Change%']) * ds['ETF Price'] / 100
ds['Arbitrage in $'] = abs(ds['Arbitrage in $'])
ds['Flag'] = 0
ds.loc[(ds['Arbitrage in $'] > ds['ETF Trading Spread in $']) & ds['ETF Trading Spread in $'] != 0, 'Flag'] = 111

In [55]:
rslt = pd.DataFrame(np.zeros((0,5)), columns=['top1','top2','top3','top4','top5'])
x=tradePricesDFMinutes.T
for i in x.columns:
    top=x.nlargest(5, i)[i].to_dict()
    topRes=pd.DataFrame([(k,v) for k,v in top.items()],index=['top1','top2','top3','top4','top5']).T
    rslt = pd.concat([rslt, topRes], axis=0)

In [58]:
[{k,v} for k,v in top.items()]

[{0.7996845030352917, 'AKAM'},
 {0.7667147282441, 'LRCX'},
 {0.593143751583769, 'AMAT'},
 {0.558759913482354, 'MU'},
 {0.5536292365802442, 'FISV'}]

In [51]:
top=x.nlargest(5, i)[i].to_dict()
l=[(k,v) for k,v in top.items()]

In [52]:
l

[('AKAM', 0.7996845030352917),
 ('LRCX', 0.7667147282441),
 ('AMAT', 0.593143751583769),
 ('MU', 0.558759913482354),
 ('FISV', 0.5536292365802442)]

In [59]:
netassetvaluereturn

Time
2020-03-17 13:30:00    0.001590
2020-03-17 13:31:00   -0.102143
2020-03-17 13:32:00    0.448515
2020-03-17 13:33:00    0.017030
2020-03-17 13:34:00   -0.445040
2020-03-17 13:35:00   -0.126553
2020-03-17 13:36:00   -0.157587
2020-03-17 13:37:00   -0.457984
2020-03-17 13:38:00    0.002403
2020-03-17 13:39:00   -0.186558
2020-03-17 13:40:00   -0.218955
2020-03-17 13:41:00    0.042534
2020-03-17 13:42:00   -0.146016
2020-03-17 13:43:00   -0.281009
2020-03-17 13:44:00   -0.225705
2020-03-17 13:45:00    0.464223
2020-03-17 13:46:00    0.460143
2020-03-17 13:47:00   -0.138171
2020-03-17 13:48:00   -0.074471
2020-03-17 13:49:00   -0.185764
2020-03-17 13:50:00   -0.378116
2020-03-17 13:51:00   -0.188970
2020-03-17 13:52:00   -0.093890
2020-03-17 13:53:00   -0.048101
2020-03-17 13:54:00   -0.040947
2020-03-17 13:55:00   -0.206190
2020-03-17 13:56:00    0.254707
2020-03-17 13:57:00    0.131894
2020-03-17 13:58:00   -0.243918
2020-03-17 13:59:00   -0.070468
2020-03-17 14:00:00   -0.261267
202